In [50]:
import scipy.io as sio
import pandas as pd
import numpy as np
import itertools

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [17]:
DIM_SIZE = 200

In [5]:
lex = sio.loadmat('data/gold_standard.mat')['gold_standard']['map']
world = sio.loadmat('data/world.mat')['world']
corpus = sio.loadmat('data/corpus.mat')['corpus']

# World

In [24]:
def get_world():
    
    world = sio.loadmat('data/world.mat')['world']
    world = world[0][0]
    
    word_idx = world[0][0]
    words = list(itertools.chain.from_iterable(map(list, world[1][0])))  
    word_dict = dict(zip(word_idx, words))
    
    obj_idx = world[3][0]
    objs = list(itertools.chain.from_iterable(map(list, world[4][0])))  
    obj_dict = dict(zip(obj_idx, objs))
    
    return word_dict, obj_dict

# Gold Standard Lexicon

In [7]:
def get_gold_lexicon(word_dict, obj_dict):
    
    lex = sio.loadmat('data/gold_standard.mat')['gold_standard']['map']
    lex = lex[0][0]
    
    lex_dict_index = {}
    lex_dict_readable = {}
    for i, j in zip(lex[0], lex[1]):
        lex_dict_index[i] = j
        lex_dict_readable[word_dict[i]] = obj_dict[j]
        
    return lex_dict_index, lex_dict_readable

# Corpus


In [8]:
def get_corpus(word_dict, obj_dict):
    
    corpus = sio.loadmat('data/corpus.mat')['corpus']
    corpus = corpus[0]
    
    corpus_idx = []
    corpus_readable = []
    for pair in corpus:
        
        scene = list(pair[0][0])
        utt = list(pair[1][0])
        new_pair = {'scene': scene, 'utt': utt}
        corpus_idx.append(new_pair)
        
        scene_word = list(map(lambda x: obj_dict[x], scene))
        utt_word = list(map(lambda x: word_dict[x], utt))
        new_pair_readable = {'scene': scene_word, 'utt': utt_word}
        corpus_readable.append(new_pair_readable)
    
    return corpus_idx, corpus_readable

In [25]:
word_dict, obj_dict = get_world()
gold_idx, gold_readable = get_gold_lexicon(word_dict, obj_dict)
corpus_idx, corpus_readable = get_corpus(word_dict, obj_dict)

In [18]:
def get_alignments(pair):
    utt, scene = pair['utt'], pair['scene']
    combos = list(itertools.product(utt, scene))
    return combos

# Neural Network Functions

In [ ]:
def apl_loss():
    return torch.max() #TODO: make this vectorized, so i=nstead of max ocompared to 0, has to be max compares to torch.zeros)

def asl_loss():
    

def joint_loss():
    pol_loss = apl_loss()
    syn_loss = asl_loss()
    return torch.add(pol_loss, syn_loss)

In [69]:
class EmbeddingModeler(nn.Module):

    def __init__(self, vocab_len, embedding_size):
        super(EmbeddingModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_len, embedding_size)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        return embeds

In [70]:
def create_embeddings(vocab_len, obj_len):
    # Access with torch.LongTensor()
    # word_embeddings(torch.LongTensor([0, 1, 2])).shape = (3, 200)
    word_embeddings = EmbeddingModeler(vocab_len, DIM_SIZE)
    obj_embeddings = EmbeddingModeler(obj_len, DIM_SIZE)
    return word_embeddings, obj_embeddings

In [19]:
vocab_len = len(word_dict)
obj_len = len(obj_dict)

In [46]:
def train_embeddings(corpus_idx, corpus_readable, vocab_len, obj_len, loss_type):
    
    word_embeddings, obj_embeddings = create_embeddings(vocab_len, obj_len)
    
    for i, pair in enumerate(corpus_idx):
        
        if loss_type == 'anti-polysemy':
            loss = apl_loss(word_embeddings, obj_embeddings)
        elif loss_type == 'anti-synonymy':
            loss = asl_loss(word_embeddings, obj_embeddings)
        else:
            loss = joint_loss(word_embeddings, obj_embeddings)

        loss.backward()
    
    return loss
    
    

In [ ]:
train_embeddings()

In [66]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
#         print(self.embeddings.weight.shape)
        embeds = self.embeddings(inputs).view((1, -1))
#         print(embeds.shape)
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)


[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [68]:
for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
#         print(context_idxs)
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
#         print(log_probs.shape)
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The los

[515.5346808433533, 513.2982707023621, 511.07346773147583, 508.8592507839203, 506.6559977531433, 504.46337628364563, 502.276953458786, 500.094162940979, 497.91767406463623, 495.74654150009155, 493.5804042816162]
